# Rapport Education

### Dates setting

In [1]:
start_date = "2023-10-01"
end_date = "2024-03-31"

In [4]:
! pip install python-dotenv
! pip install selenium
! pip install webdriver-manager
! pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.3 MB/s eta 0:00:00


### Import packages

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import warnings
warnings.filterwarnings('ignore')
from dateutil.parser import parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
from dotenv import load_dotenv
import plotly.express as px
import openpyxl

### Download files from Commcare

In [7]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('EMAIL')
password_cc = os.getenv('PASSWORD')

#Defining the driver
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(1000)

#Creating login function
def education_beneficiaries():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/d5a1d9e3e30cf72fb48f19ac18a7ec67/'
    )
    driver.find_element("xpath", '//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element("xpath", '//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR, 'button[type=submit]').click()

#Muso beneficiaries
education_beneficiaries()

#Download the database "beneficiaries"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-----------------------------------------------------------------------------------------------------------------------

#Groupes d'éducation
def education_groups():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/5cd1b95bf4b9771709f3d3d5b2efbb6e/'
    )

education_groups()

#Download the database "Household count"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-----------------------------------------------------------------------------------------------------------------------

#Session éducation
def education_session():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/4a7aef979ea6b067e3c53ad097757b8f/'
    )

education_session()

#Download the database "Household count"
driver.find_element("xpath", '//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element("xpath", '//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

False

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/131.0.6778.87/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5c2c8da9234a <unknown>
#1 0x5c2c8d5a86e0 <unknown>
#2 0x5c2c8d5dfdc8 <unknown>
#3 0x5c2c8d5db926 <unknown>
#4 0x5c2c8d6276f6 <unknown>
#5 0x5c2c8d626d46 <unknown>
#6 0x5c2c8d61b203 <unknown>
#7 0x5c2c8d5e9cc0 <unknown>
#8 0x5c2c8d5eac9e <unknown>
#9 0x5c2c8da5fd3b <unknown>
#10 0x5c2c8da63cc2 <unknown>
#11 0x5c2c8da4cb6c <unknown>
#12 0x5c2c8da64837 <unknown>
#13 0x5c2c8da3210f <unknown>
#14 0x5c2c8da815a8 <unknown>
#15 0x5c2c8da81770 <unknown>
#16 0x5c2c8da911c6 <unknown>
#17 0x7b00b7175ac3 <unknown>


### Importing datasets

In [ ]:
session1 = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/education_presence {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)
groupe = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/education_groupes_latest {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)
members = pd.read_excel(f"C:/Users/Davidson Adrien/Downloads/education_members (created 2022-03-30) {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", parse_dates = True)

## Data wrangling

In [ ]:
session = session1.copy()

#===========================================================================================================================
session.drop(['number', 'closed_by_username', 'closed_date', 'last_modified_by_user_username', 'last_modified_date',
              'opened_by_username', 'opened_date', 'owner_name', 'case_link', 'case_type', 'closed_by_user_id',
              'external_id', 'last_modified_by_user_id', 'opened_by_user_id', 'owner_id', 'server_last_modified_date',
              'state', 'chlore_or_abendazole'], axis = 1, inplace = True)

#==========================================================================================================================
group = groupe[['caseid', 'group_commune', 'groupe_type', 'closed']]
member = members[['caseid', 'dob', 'age', 'sexe', 'closed']]

#==========================================================================================================================

member.rename(columns = {'caseid': 'member_case_id',
                        'closed': 'closed_member',
                        'indices.education_groupes' : 'group_caseid'}, inplace = True)
session.rename(columns = {'indices.education_groupes': 'group_caseid',
                         'closed': 'closed_session'}, inplace = True)
group.rename(columns = {'caseid': 'group_caseid',
                       'closed': 'closed_group'}, inplace = True)
group.replace(['Les Cayes'], ['Cayes'], inplace = True)
#======================================================================================================================

session = session.merge(member, on = 'member_case_id', how = 'left').merge(group, on = 'group_caseid', how = 'left')
session.loc[(session['sexe'] == 0), 'sexe'] = 'Female'
session.loc[(session['sexe'] == 1), 'sexe'] = 'Male'


In [ ]:
session.columns

Index(['caseid', 'name', 'date_de_presence', 'is_member_present',
       'member_case_id', 'group_caseid', 'closed_session', 'dob', 'age',
       'sexe', 'closed_member', 'group_commune', 'groupe_type',
       'closed_group'],
      dtype='object')

In [ ]:
# session['dob'].replace("NaT", "0000-00-00", inplace = True)
session['dob'] = pd.to_datetime(session['dob'], errors = 'coerce')

def get_age(date):
    if pd.notna(date):
        return (parse(end_date) - date)/365.25
    else:
        return "---"

session['age_new'] = session['dob'].map(get_age)
session['age_new'] = session['age'].where(session['age_new'] == "---", other=session['age_new'])
session['age_new'].fillna(0, inplace=True)
session['age_new']

0       12 days 13:55:48.665297741
1       10 days 19:01:21.314168377
2       12 days 05:31:10.225872690
3       13 days 17:08:00.492813141
4       10 days 08:06:53.963039014
                   ...            
31300   10 days 05:52:51.252566735
31301   10 days 05:52:51.252566735
31302   10 days 05:52:51.252566735
31303   13 days 05:53:50.390143737
31304   10 days 05:52:51.252566735
Name: age_new, Length: 31305, dtype: timedelta64[ns]

In [ ]:
session['age_new'] = pd.to_numeric(session['age_new'], errors="coerce")

In [ ]:
def age_range(age):
    if age in range(10, 15):
        return '10 - 14'
    else:
        return 'other'

session['age_range'] = session['age_new'].map(age_range)
session['age_range']

0        other
1        other
2        other
3        other
4        other
         ...  
31300    other
31301    other
31302    other
31303    other
31304    other
Name: age_range, Length: 31305, dtype: object

In [ ]:
session['age_range'].unique()

array(['other'], dtype=object)

In [ ]:
session.to_excel("Education_FY24Q2_04122024.xlsx", index = False)

In [ ]:
session_fy23 = session[(session['date_de_presence'] >= start_date) & (session['date_de_presence'] <= end_date)]

In [ ]:
comm = session_fy23[session_fy23['groupe_type'] == 'community_club']
school = session_fy23[session_fy23['groupe_type'] == 'school']

### Community clubs information

In [ ]:
comm_pres = comm[comm['is_member_present'] == 1]

comm.shape[0]
comm_pres.shape[0]

52107

43665

In [ ]:
session_fy23.groupe_type.unique()

array(['community_club', 'school'], dtype=object)

In [ ]:
comprehensive1 = comprehensive.groupby(['group_commune', 'member_case_id'])['is_member_present'].count()
comprehensive1 = comprehensive1.reset_index()
comprehensive1.rename(columns = {'is_member_present': 'presence'}, inplace = True)
comprehensive1 = comprehensive1.merge(comprehensive[['member_case_id', 'dob', 'age', 'sexe', 'status']], on = 'member_case_id', how = 'left')
comprehensive1['age'] = pd.to_numeric(comprehensive1['age'], errors = 'coerce')
comprehensive1 = comprehensive1.drop_duplicates('member_case_id').reset_index(drop=True)
comprehensive1 = comprehensive1[comprehensive1['presence'] >= 8]

#=================================================================================================================

def age_range(row):
    if row['age'] < 1:
        return '< 1'
    elif row['age'] in range(1, 5):
        return '1 - 4'
    elif row['age'] in range(5, 10):
        return '5 - 9'
    elif row['age'] in range(10, 15):
        return '10 - 14'
    elif row['age'] in range(15, 18):
        return '15 - 17'
    elif row['age'] in range(18, 21):
        return '18 - 20'
    else:
        return '21+'

comprehensive1['age_range'] = comprehensive1.apply(age_range, axis =1)
#=================================================================================================================
#=================================================================================================================
comprehensive1.head()

,group_commune,member_case_id,presence,dob,age,sexe,status,age_range
0,Cap-Haitien,0c08b71d373344689193b463e8661498,9,2010-09-18,12.0,Female,test not indicated,10 - 14
1,Cap-Haitien,0e11112d1a914caa9bfd7f96e45f99c5,9,2008-09-18,14.0,Female,test not indicated,10 - 14
2,Cap-Haitien,0e8fa9aa856f4b15b92d677138203fdc,9,2010-09-18,12.0,Female,test not indicated,10 - 14
3,Cap-Haitien,0ed91047f6844a718148bc81c5ce9737,9,2010-09-18,12.0,Female,test not indicated,10 - 14
4,Cap-Haitien,15dc64ef7a3d4a86b03c49213600b46c,9,2012-09-17,10.0,Male,test not indicated,10 - 14


In [ ]:
comprehensive1.to_excel('Beneficiaires_CC_Q2FY23_04202023.xlsx')

In [ ]:
datim1 = pd.pivot_table(comprehensive1, values = 'member_case_id', index = 'group_commune',
                        columns = ['sexe', 'age_range'], aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')

columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18 - 20'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18 - 20')]

datim1 = datim1.reindex(columns = columns)
datim1[('Female', '18 - 20')] = 0
datim1[('Male', '18 - 20')] = 0
datim1.fillna(0, inplace = True)
datim1.reset_index(inplace = True)
datim1

sexe        group_commune Female                                     Male  \
age_range                    < 1 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20  < 1   
0             Cap-Haitien    0.0   0.0     1      16       3       0  0.0   
1             Cap-Haïtien    0.0   0.0    19      97      53       0  0.0   
2                   Cayes    0.0   0.0    39     270     197       0  0.0   
3                  Delmas    0.0   0.0     8      47       9       0  0.0   
4                  Dondon    0.0   0.0     1       5       0       0  0.0   
5                   Ducis    0.0   0.0    22      77      31       0  0.0   
6                Gonaïves    0.0   0.0     6      19       4       0  0.0   
7              Gros-Morne    0.0   0.0     1      12      10       0  0.0   
8                  Guilgo    0.0   0.0     0      35       8       0  0.0   
9                 Jérémie    0.0   0.0     6      63      25       0  0.0   
10                  Limbé    0.0   0.0    14      93      15       0  0.0   
11             Port-Salut    0.0   0.0    12     295     144       0  0.0   
12         Port-au-Prince    0.0   0.0    10      96      53       0  0.0   
13                Tabarre    0.0   0.0     3       9       1       0  0.0   
14                  Total    0.0   0.0   142    1134     553       0  0.0   

sexe                                           
age_range 1 - 4 5 - 9 10 - 14 15 - 17 18 - 20  
0           0.0     0       6       1       0  
1           0.0    12      86      33       0  
2           0.0    19     117      55       0  
3           0.0     5      31       9       0  
4           0.0     3      11       0       0  
5           0.0    18      42      11       0  
6           0.0     4      14       1       0  
7           0.0     0      16       5       0  
8           0.0     0       6       6       0  
9           0.0     5      54      12       0  
10          0.0    18      78       9       0  
11          0.0     0      28      16       0  
12          0.0     5      37       6       0  
13          0.0     4       4       5       0  
14          0.0    93     530     169       0

In [ ]:
driver.quit()